In [ ]:
import os
import glob
import time
import torch
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset

import torchvision.models as models
from torchvision import transforms, utils

from PIL import Image
from skimage.transform import resize

import warnings
warnings.filterwarnings("ignore")

### Load Dataset

In [ ]:
class dataset(Dataset):
    def __init__(self, root_dir, dataset_type, img_size, transform=None, shuffle=False):
        self.root_dir = root_dir
        self.transform = transform
        self.file_names = [f for f in glob.glob(os.path.join(root_dir, "*", "*.npz")) \
                            if dataset_type in f]
        self.img_size = img_size
        #self.embeddings = np.load(os.path.join(root_dir, 'embedding.npy'), allow_pickle=True,encoding ='latin1')
        self.shuffle = shuffle

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        data_path = self.file_names[idx]
        data = np.load(data_path)
        image = data["image"].reshape(16, 160, 160)
        target = data["target"]
        structure = data["structure"]
        meta_target = data["meta_target"]
        meta_structure = data["meta_structure"]

        if self.shuffle:
            context = image[:8, :, :]
            choices = image[8:, :, :]
            indices = list(range(8))
            np.random.shuffle(indices)
            new_target = indices.index(target)
            new_choices = choices[indices, :, :]
            image = np.concatenate((context, new_choices))
            target = new_target
        
        resize_image = []
        for idx in range(0, 16):
            resize_image.append(resize(image[idx,:,:], (self.img_size, self.img_size)))
        resize_image = np.stack(resize_image)

        embedding = torch.zeros((6, 300), dtype=torch.float)
        indicator = torch.zeros(1, dtype=torch.float)
        element_idx = 0
    
        del data
        if self.transform:
            resize_image = self.transform(resize_image)
            # meta_matrix = self.transform(meta_matrix)
            target = torch.tensor(target, dtype=torch.long)
            meta_target = self.transform(meta_target)
            meta_structure = self.transform(meta_structure)
            # meta_target = torch.tensor(meta_target, dtype=torch.long)
        return resize_image, target, meta_target, meta_structure, embedding, indicator

#### Experiment Hyperparameters

In [ ]:
class Args:
    
    def __init__(self,):
        self.model = 'Resnet18_MLP'
        self.epochs = 100
        self.batch_size = 32
        self.seed = 12345
        self.device = 0
        self.load_workers = 80
        self.resume = False
        self.path = '/common/home/ls1274/Downloads'
        self.save = './ckpt_res/'
        self.img_size = 224
        self.lr = 1e-4
        self.beta1 = 0.9
        self.beta2 = 0.999
        self.epsilon = 1e-8
        self.meta_alpha = 0.
        self.meta_beta = 0.
        self.perc_train = 100
        
args = Args()

args.cuda = torch.cuda.is_available()
torch.cuda.set_device(args.device)
torch.cuda.manual_seed(args.seed)

if not os.path.exists(args.save):
    os.makedirs(args.save)

#### Load Data

In [ ]:
def gen():
    file_list = []
    for root, dir, files in os.walk("/common/home/kp956/CS536/I-RAVEN/test/"):
        for file in files:
            if file.endswith(".npz"):
                file_list.append(os.path.join(root,file))
    return file_list

class ToTensor(object):
    def __call__(self, sample):
        return torch.tensor(sample, dtype=torch.float32)

train = dataset(args.path, "train", args.img_size, transform=transforms.Compose([ToTensor()]),shuffle=True)
valid = dataset(args.path, "val", args.img_size, transform=transforms.Compose([ToTensor()]))
test = dataset(args.path, "test", args.img_size, transform=transforms.Compose([ToTensor()]))

subset_indices = np.random.choice(len(train), len(train)*args.perc_train // 100, replace=False)
train_subset = Subset(train, subset_indices)

print("Number of samples in original train set =", len(train))
print("Number of samples in train subset =", len(train_subset))
print("All samples are unique =", len(subset_indices) == len(set(subset_indices)))

trainloader = DataLoader(train_subset, batch_size=args.batch_size, shuffle=False, num_workers=80)
validloader = DataLoader(valid, batch_size=args.batch_size, shuffle=False, num_workers=80)
testloader = DataLoader(test, batch_size=args.batch_size, shuffle=False, num_workers=80)

Number of samples in original train set = 24000
Number of samples in train subset = 24000
All samples are unique = True


### Model

In [ ]:
class BasicModel(nn.Module):
    def __init__(self, args):
        super(BasicModel, self).__init__()
        self.name = args.model
    
    def load_model(self, path, epoch):
        state_dict = torch.load(path+'{}_epoch_{}.pth'.format(self.name, epoch))['state_dict']
        self.load_state_dict(state_dict)

    def save_model(self, path, epoch, acc, loss):
        torch.save({'state_dict': self.state_dict(), 'acc': acc, 'loss': loss}, path+'{}_epoch_{}.pth'.format(self.name, epoch))

    def compute_loss(self, output, target, meta_target, meta_structure):
        pass

    def train_(self, image, target, meta_target, meta_structure, embedding, indicator):
        self.optimizer.zero_grad()
        output = self(image, embedding, indicator)
        loss = self.compute_loss(output, target, meta_target, meta_structure)
        loss.backward()
        self.optimizer.step()
        pred = output[0].data.max(1)[1]
        correct = pred.eq(target.data).cpu().sum().numpy()
        accuracy = correct * 100.0 / target.size()[0]
        return loss.item(), accuracy

    def validate_(self, image, target, meta_target, meta_structure, embedding, indicator):
        with torch.no_grad():
            output = self(image, embedding, indicator)
        loss = self.compute_loss(output, target, meta_target, meta_structure)
        pred = output[0].data.max(1)[1]
        correct = pred.eq(target.data).cpu().sum().numpy()
        accuracy = correct * 100.0 / target.size()[0]
        return loss.item(), accuracy

    def test_(self, image, target, meta_target, meta_structure, embedding, indicator):
        with torch.no_grad():
            output = self(image, embedding, indicator)
        pred = output[0].data.max(1)[1]
        correct = pred.eq(target.data).cpu().sum().numpy()
        accuracy = correct * 100.0 / target.size()[0]
        return accuracy

#### ResNet18

In [ ]:
class identity(nn.Module):
    def __init__(self):
        super(identity, self).__init__()
    
    def forward(self, x):
        return x

class mlp_module(nn.Module):
    def __init__(self):
        super(mlp_module, self).__init__()
        self.fc1 = nn.Linear(512, 512)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(512, 8+9+21)
        self.dropout = nn.Dropout(0.5)
        self.sf = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.sf(x)
        return x

class Resnet18_MLP(BasicModel):
    def __init__(self, args):
        super(Resnet18_MLP, self).__init__(args)
        self.resnet18 = models.resnet18(pretrained=False)
        self.resnet18.conv1 = nn.Conv2d(16, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.resnet18.fc = identity()
        self.mlp = mlp_module()
        self.optimizer = optim.Adam(self.parameters(), lr=args.lr, betas=(args.beta1, args.beta2), eps=args.epsilon)
        self.meta_alpha = args.meta_alpha
        self.meta_beta = args.meta_beta

    def compute_loss(self, output, target, meta_target, meta_structure):
        pred, meta_target_pred, meta_struct_pred = output[0], output[1], output[2]

        target_loss = F.cross_entropy(pred, target)
        loss = target_loss
        return loss

    def forward(self, x, embedding, indicator):
        alpha = 0
        features = self.resnet18(x.view(-1, 16, 224, 224))
        features_tree = features.view(-1, 1, 512)
        final_features = features + alpha #* features_tree
        output = self.mlp(final_features)
        pred = output[:,0:8]
        meta_target_pred = output[:,8:17]
        meta_struct_pred = output[:,17:38]
        return pred, meta_target_pred, meta_struct_pred

### Training and Evalution

In [ ]:
### Helper functions

def train(epoch, save_file):
    model.train()
    train_loss = 0
    accuracy = 0

    loss_all = 0.0
    acc_all = 0.0
    counter = 0
    for batch_idx, (image, target, meta_target, meta_structure, embedding, indicator) in enumerate(trainloader):
        counter += 1
        if args.cuda:
            image = image.cuda()
            target = target.cuda()
            meta_target = meta_target.cuda()
            meta_structure = meta_structure.cuda()
            embedding = embedding.cuda()
            indicator = indicator.cuda()
        loss, acc = model.train_(image, target, meta_target, meta_structure, embedding, indicator)
        save_str = 'Train: Epoch:{}, Batch:{}, Loss:{:.6f}, Acc:{:.4f}'.format(epoch, batch_idx, loss, acc)
        if counter % 20 == 0:
            print(save_str)
        with open(save_file, 'a') as f:
            f.write(save_str + "\n")
        loss_all += loss
        acc_all += acc
    if counter > 0:
        save_str = "Train_: Avg Training Loss: {:.6f}, Avg Training Acc: {:.6f}".format(
            loss_all/float(counter),
            acc_all/float(counter)
        )
        print(save_str)
        with open(save_file, 'a') as f:
            f.write(save_str + "\n")
    return loss_all/float(counter), acc_all/float(counter)

def validate(epoch, save_file):
    model.eval()
    val_loss = 0
    accuracy = 0

    loss_all = 0.0
    acc_all = 0.0
    counter = 0
    batch_idx = 0
    for batch_idx, (image, target, meta_target, meta_structure, embedding, indicator) in enumerate(validloader):
        #print(image)
        counter += 1
        if args.cuda:
            image = image.cuda()
            target = target.cuda()
            meta_target = meta_target.cuda()
            meta_structure = meta_structure.cuda()
            embedding = embedding.cuda()
            indicator = indicator.cuda()
        loss, acc = model.validate_(image, target, meta_target, meta_structure, embedding, indicator)
        # print('Validate: Epoch:{}, Batch:{}, Loss:{:.6f}, Acc:{:.4f}.'.format(epoch, batch_idx, loss, acc)) 
        loss_all += loss
        acc_all += acc
    if counter > 0:
        save_str = "Val_: Total Validation Loss: {:.6f}, Acc: {:.4f}".format(loss_all/float(counter), acc_all/float(counter))
        print(save_str)
        with open(save_file, 'a') as f:
            f.write(save_str + "\n")
    return loss_all/float(counter), acc_all/float(counter)

def test(epoch, save_file):
    model.eval()
    accuracy = 0

    acc_all = 0.0
    counter = 0
    for batch_idx, (image, target, meta_target, meta_structure, embedding, indicator) in enumerate(testloader):
        counter += 1
        if args.cuda:
            image = image.cuda()
            target = target.cuda()
            meta_target = meta_target.cuda()
            meta_structure = meta_structure.cuda()
            embedding = embedding.cuda()
            indicator = indicator.cuda()
        acc = model.test_(image, target, meta_target, meta_structure, embedding, indicator)
        # print('Test: Epoch:{}, Batch:{}, Acc:{:.4f}.'.format(epoch, batch_idx, acc))  
        acc_all += acc
    if counter > 0:
        save_str = "Test_: Total Testing Acc: {:.4f}".format(acc_all / float(counter))
        print(save_str)
        with open(save_file, 'a') as f:
            f.write(save_str + "\n")
    return acc_all/float(counter)

In [ ]:
model = Resnet18_MLP(args)
model = model.cuda()

In [ ]:
SAVE_FILE = "new_resnet_100dat_100eps" + time.strftime("%Y-%m-%d_%H:%M:%S", time.gmtime()) + "_" + str(args.perc_train)

for epoch in range(0, args.epochs):
    t0 = time.time()
    train(epoch, SAVE_FILE)
    avg_loss, avg_acc = validate(epoch, SAVE_FILE)
    test_acc = test(epoch, SAVE_FILE)
    model.save_model(args.save, epoch, avg_acc, avg_loss)
    print("Time taken = {:.4f} s\n".format(time.time() - t0))

Train: Epoch:0, Batch:19, Loss:2.080635, Acc:9.3750
Train: Epoch:0, Batch:39, Loss:2.082202, Acc:12.5000
Train: Epoch:0, Batch:59, Loss:2.079495, Acc:15.6250
Train: Epoch:0, Batch:79, Loss:2.078242, Acc:15.6250
Train: Epoch:0, Batch:99, Loss:2.078342, Acc:15.6250
Train: Epoch:0, Batch:119, Loss:2.069187, Acc:21.8750
Train: Epoch:0, Batch:139, Loss:2.102238, Acc:15.6250
Train: Epoch:0, Batch:159, Loss:2.031045, Acc:25.0000
Train: Epoch:0, Batch:179, Loss:2.081956, Acc:12.5000
Train: Epoch:0, Batch:199, Loss:2.087350, Acc:9.3750
Train: Epoch:0, Batch:219, Loss:2.049453, Acc:15.6250
Train: Epoch:0, Batch:239, Loss:2.027860, Acc:21.8750
Train: Epoch:0, Batch:259, Loss:2.102485, Acc:15.6250
Train: Epoch:0, Batch:279, Loss:2.039191, Acc:21.8750
Train: Epoch:0, Batch:299, Loss:2.054290, Acc:18.7500
Train: Epoch:0, Batch:319, Loss:2.074665, Acc:12.5000
Train: Epoch:0, Batch:339, Loss:2.007359, Acc:28.1250
Train: Epoch:0, Batch:359, Loss:2.052141, Acc:9.3750
Train: Epoch:0, Batch:379, Loss:2.04

Train: Epoch:3, Batch:659, Loss:2.048050, Acc:15.6250
Train: Epoch:3, Batch:679, Loss:1.967530, Acc:28.1250
Train: Epoch:3, Batch:699, Loss:2.072402, Acc:15.6250
Train: Epoch:3, Batch:719, Loss:2.132700, Acc:0.0000
Train: Epoch:3, Batch:739, Loss:2.102537, Acc:12.5000
Train_: Avg Training Loss: 2.024393, Avg Training Acc: 20.279167
Val_: Total Validation Loss: 2.019209, Acc: 20.4875
Test_: Total Testing Acc: 20.1250
Time taken = 222.3962 s

Train: Epoch:4, Batch:19, Loss:2.059431, Acc:18.7500
Train: Epoch:4, Batch:39, Loss:2.018404, Acc:25.0000
Train: Epoch:4, Batch:59, Loss:2.035155, Acc:15.6250
Train: Epoch:4, Batch:79, Loss:2.020538, Acc:18.7500
Train: Epoch:4, Batch:99, Loss:2.006946, Acc:21.8750
Train: Epoch:4, Batch:119, Loss:1.952959, Acc:28.1250
Train: Epoch:4, Batch:139, Loss:2.071788, Acc:15.6250
Train: Epoch:4, Batch:159, Loss:1.985968, Acc:21.8750
Train: Epoch:4, Batch:179, Loss:2.054232, Acc:15.6250
Train: Epoch:4, Batch:199, Loss:2.020956, Acc:21.8750
Train: Epoch:4, Batc

Train: Epoch:7, Batch:499, Loss:2.061190, Acc:18.7500
Train: Epoch:7, Batch:519, Loss:1.943288, Acc:31.2500
Train: Epoch:7, Batch:539, Loss:1.959282, Acc:28.1250
Train: Epoch:7, Batch:559, Loss:1.985904, Acc:21.8750
Train: Epoch:7, Batch:579, Loss:2.065051, Acc:18.7500
Train: Epoch:7, Batch:599, Loss:2.003492, Acc:18.7500
Train: Epoch:7, Batch:619, Loss:1.950036, Acc:25.0000
Train: Epoch:7, Batch:639, Loss:2.001985, Acc:21.8750
Train: Epoch:7, Batch:659, Loss:2.010885, Acc:15.6250
Train: Epoch:7, Batch:679, Loss:1.972032, Acc:21.8750
Train: Epoch:7, Batch:699, Loss:2.074619, Acc:15.6250
Train: Epoch:7, Batch:719, Loss:2.118464, Acc:3.1250
Train: Epoch:7, Batch:739, Loss:2.089989, Acc:9.3750
Train_: Avg Training Loss: 2.013979, Avg Training Acc: 21.462500
Val_: Total Validation Loss: 2.016264, Acc: 22.0750
Test_: Total Testing Acc: 21.6500
Time taken = 203.5610 s

Train: Epoch:8, Batch:19, Loss:2.086435, Acc:12.5000
Train: Epoch:8, Batch:39, Loss:1.986262, Acc:28.1250
Train: Epoch:8, Ba

Train: Epoch:11, Batch:319, Loss:1.967265, Acc:28.1250
Train: Epoch:11, Batch:339, Loss:1.865290, Acc:37.5000
Train: Epoch:11, Batch:359, Loss:1.985223, Acc:28.1250
Train: Epoch:11, Batch:379, Loss:1.908892, Acc:31.2500
Train: Epoch:11, Batch:399, Loss:1.965001, Acc:18.7500
Train: Epoch:11, Batch:419, Loss:2.077762, Acc:12.5000
Train: Epoch:11, Batch:439, Loss:2.060509, Acc:15.6250
Train: Epoch:11, Batch:459, Loss:1.934173, Acc:28.1250
Train: Epoch:11, Batch:479, Loss:2.012326, Acc:28.1250
Train: Epoch:11, Batch:499, Loss:2.104353, Acc:6.2500
Train: Epoch:11, Batch:519, Loss:1.869497, Acc:46.8750
Train: Epoch:11, Batch:539, Loss:1.848308, Acc:43.7500
Train: Epoch:11, Batch:559, Loss:1.941051, Acc:28.1250
Train: Epoch:11, Batch:579, Loss:1.981438, Acc:21.8750
Train: Epoch:11, Batch:599, Loss:1.969313, Acc:31.2500
Train: Epoch:11, Batch:619, Loss:1.981954, Acc:25.0000
Train: Epoch:11, Batch:639, Loss:2.019287, Acc:21.8750
Train: Epoch:11, Batch:659, Loss:1.981320, Acc:21.8750
Train: Epoc

Train: Epoch:15, Batch:99, Loss:1.936646, Acc:25.0000
Train: Epoch:15, Batch:119, Loss:1.965721, Acc:25.0000
Train: Epoch:15, Batch:139, Loss:2.036400, Acc:15.6250
Train: Epoch:15, Batch:159, Loss:1.944067, Acc:28.1250
Train: Epoch:15, Batch:179, Loss:2.047061, Acc:15.6250
Train: Epoch:15, Batch:199, Loss:2.027387, Acc:15.6250
Train: Epoch:15, Batch:219, Loss:1.860687, Acc:46.8750
Train: Epoch:15, Batch:239, Loss:1.849076, Acc:40.6250
Train: Epoch:15, Batch:259, Loss:1.986065, Acc:28.1250
Train: Epoch:15, Batch:279, Loss:1.975160, Acc:25.0000
Train: Epoch:15, Batch:299, Loss:1.931489, Acc:40.6250
Train: Epoch:15, Batch:319, Loss:1.900986, Acc:31.2500
Train: Epoch:15, Batch:339, Loss:1.900983, Acc:28.1250
Train: Epoch:15, Batch:359, Loss:2.031436, Acc:25.0000
Train: Epoch:15, Batch:379, Loss:2.050028, Acc:28.1250
Train: Epoch:15, Batch:399, Loss:1.951895, Acc:21.8750
Train: Epoch:15, Batch:419, Loss:2.032610, Acc:15.6250
Train: Epoch:15, Batch:439, Loss:2.020555, Acc:18.7500
Train: Epoc

...

Train: Epoch:82, Batch:139, Loss:1.431379, Acc:84.3750
Train: Epoch:82, Batch:159, Loss:1.574234, Acc:68.7500
Train: Epoch:82, Batch:179, Loss:1.372102, Acc:90.6250
Train: Epoch:82, Batch:199, Loss:1.472008, Acc:81.2500
Train: Epoch:82, Batch:219, Loss:1.428788, Acc:84.3750
Train: Epoch:82, Batch:239, Loss:1.368453, Acc:90.6250
Train: Epoch:82, Batch:259, Loss:1.447261, Acc:84.3750
Train: Epoch:82, Batch:279, Loss:1.462085, Acc:81.2500
Train: Epoch:82, Batch:299, Loss:1.356536, Acc:90.6250
Train: Epoch:82, Batch:319, Loss:1.525299, Acc:75.0000
Train: Epoch:82, Batch:339, Loss:1.441393, Acc:84.3750
Train: Epoch:82, Batch:359, Loss:1.325877, Acc:93.7500
Train: Epoch:82, Batch:379, Loss:1.370945, Acc:90.6250
Train: Epoch:82, Batch:399, Loss:1.491710, Acc:78.1250
Train: Epoch:82, Batch:419, Loss:1.477312, Acc:78.1250
Train: Epoch:82, Batch:439, Loss:1.462441, Acc:81.2500
Train: Epoch:82, Batch:459, Loss:1.429913, Acc:84.3750
Train: Epoch:82, Batch:479, Loss:1.527332, Acc:75.0000
Train: Epo

Train: Epoch:85, Batch:719, Loss:1.670415, Acc:59.3750
Train: Epoch:85, Batch:739, Loss:1.580537, Acc:68.7500
Train_: Avg Training Loss: 1.437549, Avg Training Acc: 83.670833
Val_: Total Validation Loss: 1.823115, Acc: 44.4875
Test_: Total Testing Acc: 43.3750
Time taken = 214.3893 s

Train: Epoch:86, Batch:19, Loss:1.465657, Acc:81.2500
Train: Epoch:86, Batch:39, Loss:1.404946, Acc:87.5000
Train: Epoch:86, Batch:59, Loss:1.518528, Acc:75.0000
Train: Epoch:86, Batch:79, Loss:1.552018, Acc:71.8750
Train: Epoch:86, Batch:99, Loss:1.386275, Acc:87.5000
Train: Epoch:86, Batch:119, Loss:1.456883, Acc:81.2500
Train: Epoch:86, Batch:139, Loss:1.461921, Acc:81.2500
Train: Epoch:86, Batch:159, Loss:1.524720, Acc:75.0000
Train: Epoch:86, Batch:179, Loss:1.458460, Acc:81.2500
Train: Epoch:86, Batch:199, Loss:1.530163, Acc:71.8750
Train: Epoch:86, Batch:219, Loss:1.462147, Acc:81.2500
Train: Epoch:86, Batch:239, Loss:1.400423, Acc:87.5000
Train: Epoch:86, Batch:259, Loss:1.459731, Acc:78.1250
Trai

Train: Epoch:89, Batch:499, Loss:1.492063, Acc:78.1250
Train: Epoch:89, Batch:519, Loss:1.343203, Acc:93.7500
Train: Epoch:89, Batch:539, Loss:1.368135, Acc:90.6250
Train: Epoch:89, Batch:559, Loss:1.429464, Acc:84.3750
Train: Epoch:89, Batch:579, Loss:1.451301, Acc:81.2500
Train: Epoch:89, Batch:599, Loss:1.492202, Acc:78.1250
Train: Epoch:89, Batch:619, Loss:1.448008, Acc:81.2500
Train: Epoch:89, Batch:639, Loss:1.460863, Acc:81.2500
Train: Epoch:89, Batch:659, Loss:1.346886, Acc:93.7500
Train: Epoch:89, Batch:679, Loss:1.432406, Acc:84.3750
Train: Epoch:89, Batch:699, Loss:1.424907, Acc:81.2500
Train: Epoch:89, Batch:719, Loss:1.649776, Acc:62.5000
Train: Epoch:89, Batch:739, Loss:1.599318, Acc:68.7500
Train_: Avg Training Loss: 1.429603, Avg Training Acc: 84.450000
Val_: Total Validation Loss: 1.851938, Acc: 41.5875
Test_: Total Testing Acc: 41.5250
Time taken = 227.0965 s

Train: Epoch:90, Batch:19, Loss:1.444498, Acc:84.3750
Train: Epoch:90, Batch:39, Loss:1.336694, Acc:93.7500
T

Train: Epoch:93, Batch:279, Loss:1.457540, Acc:81.2500
Train: Epoch:93, Batch:299, Loss:1.368499, Acc:90.6250
Train: Epoch:93, Batch:319, Loss:1.462376, Acc:81.2500
Train: Epoch:93, Batch:339, Loss:1.399311, Acc:87.5000
Train: Epoch:93, Batch:359, Loss:1.275935, Acc:100.0000
Train: Epoch:93, Batch:379, Loss:1.370955, Acc:90.6250
Train: Epoch:93, Batch:399, Loss:1.432422, Acc:84.3750
Train: Epoch:93, Batch:419, Loss:1.451442, Acc:81.2500
Train: Epoch:93, Batch:439, Loss:1.462917, Acc:81.2500
Train: Epoch:93, Batch:459, Loss:1.444152, Acc:84.3750
Train: Epoch:93, Batch:479, Loss:1.434627, Acc:84.3750
Train: Epoch:93, Batch:499, Loss:1.486925, Acc:78.1250
Train: Epoch:93, Batch:519, Loss:1.382710, Acc:87.5000
Train: Epoch:93, Batch:539, Loss:1.375190, Acc:90.6250
Train: Epoch:93, Batch:559, Loss:1.430504, Acc:84.3750
Train: Epoch:93, Batch:579, Loss:1.426204, Acc:84.3750
Train: Epoch:93, Batch:599, Loss:1.430955, Acc:84.3750
Train: Epoch:93, Batch:619, Loss:1.491584, Acc:78.1250
Train: Ep

Train: Epoch:97, Batch:59, Loss:1.550202, Acc:71.8750
Train: Epoch:97, Batch:79, Loss:1.462619, Acc:81.2500
Train: Epoch:97, Batch:99, Loss:1.310161, Acc:96.8750
Train: Epoch:97, Batch:119, Loss:1.376247, Acc:90.6250
Train: Epoch:97, Batch:139, Loss:1.494358, Acc:78.1250
Train: Epoch:97, Batch:159, Loss:1.465939, Acc:78.1250
Train: Epoch:97, Batch:179, Loss:1.427877, Acc:84.3750
Train: Epoch:97, Batch:199, Loss:1.461444, Acc:81.2500
Train: Epoch:97, Batch:219, Loss:1.427242, Acc:84.3750
Train: Epoch:97, Batch:239, Loss:1.349923, Acc:93.7500
Train: Epoch:97, Batch:259, Loss:1.441263, Acc:84.3750
Train: Epoch:97, Batch:279, Loss:1.413157, Acc:84.3750
Train: Epoch:97, Batch:299, Loss:1.369405, Acc:90.6250
Train: Epoch:97, Batch:319, Loss:1.442975, Acc:84.3750
Train: Epoch:97, Batch:339, Loss:1.398365, Acc:87.5000
Train: Epoch:97, Batch:359, Loss:1.276275, Acc:100.0000
Train: Epoch:97, Batch:379, Loss:1.337891, Acc:93.7500
Train: Epoch:97, Batch:399, Loss:1.452282, Acc:81.2500
Train: Epoch

In [ ]:
#Re-running the model 60% data
SAVE_FILE = "new_resnet_60dat_100eps" + time.strftime("%Y-%m-%d_%H:%M:%S", time.gmtime()) + "_" + str(args.perc_train)

for epoch in range(0, args.epochs):
    t0 = time.time()
    train(epoch, SAVE_FILE)
    avg_loss, avg_acc = validate(epoch, SAVE_FILE)
    test_acc = test(epoch, SAVE_FILE)
    model.save_model(args.save, epoch, avg_acc, avg_loss)
    print("Time taken = {:.4f} s\n".format(time.time() - t0))

Train: Epoch:0, Batch:19, Loss:2.080345, Acc:18.7500
Train: Epoch:0, Batch:39, Loss:2.080103, Acc:12.5000
Train: Epoch:0, Batch:59, Loss:2.082830, Acc:6.2500
Train: Epoch:0, Batch:79, Loss:2.058698, Acc:18.7500
Train: Epoch:0, Batch:99, Loss:2.083550, Acc:9.3750
Train: Epoch:0, Batch:119, Loss:2.080232, Acc:12.5000
Train: Epoch:0, Batch:139, Loss:2.081726, Acc:12.5000
Train: Epoch:0, Batch:159, Loss:2.081982, Acc:6.2500
Train: Epoch:0, Batch:179, Loss:2.075108, Acc:12.5000
Train: Epoch:0, Batch:199, Loss:2.077135, Acc:15.6250
Train: Epoch:0, Batch:219, Loss:2.082624, Acc:9.3750
Train: Epoch:0, Batch:239, Loss:2.073303, Acc:25.0000
Train: Epoch:0, Batch:259, Loss:2.060551, Acc:18.7500
Train: Epoch:0, Batch:279, Loss:2.073835, Acc:21.8750
Train: Epoch:0, Batch:299, Loss:2.087578, Acc:9.3750
Train: Epoch:0, Batch:319, Loss:2.086034, Acc:3.1250
Train: Epoch:0, Batch:339, Loss:2.080724, Acc:9.3750
Train: Epoch:0, Batch:359, Loss:2.066674, Acc:21.8750
Train: Epoch:0, Batch:379, Loss:2.033298

Train: Epoch:6, Batch:39, Loss:2.059095, Acc:15.6250
Train: Epoch:6, Batch:59, Loss:1.960372, Acc:28.1250
Train: Epoch:6, Batch:79, Loss:1.967380, Acc:34.3750
Train: Epoch:6, Batch:99, Loss:1.868548, Acc:40.6250
Train: Epoch:6, Batch:119, Loss:2.043218, Acc:21.8750
Train: Epoch:6, Batch:139, Loss:1.932630, Acc:25.0000
Train: Epoch:6, Batch:159, Loss:1.998248, Acc:28.1250
Train: Epoch:6, Batch:179, Loss:1.953639, Acc:25.0000
Train: Epoch:6, Batch:199, Loss:1.959143, Acc:21.8750
Train: Epoch:6, Batch:219, Loss:2.047068, Acc:15.6250
Train: Epoch:6, Batch:239, Loss:1.999524, Acc:18.7500
Train: Epoch:6, Batch:259, Loss:1.906157, Acc:31.2500
Train: Epoch:6, Batch:279, Loss:2.025283, Acc:15.6250
Train: Epoch:6, Batch:299, Loss:1.936867, Acc:28.1250
Train: Epoch:6, Batch:319, Loss:1.943050, Acc:31.2500
Train: Epoch:6, Batch:339, Loss:1.985132, Acc:31.2500
Train: Epoch:6, Batch:359, Loss:1.994002, Acc:21.8750
Train: Epoch:6, Batch:379, Loss:1.947418, Acc:25.0000
Train: Epoch:6, Batch:399, Loss:

Train: Epoch:12, Batch:39, Loss:2.036763, Acc:21.8750
Train: Epoch:12, Batch:59, Loss:1.926432, Acc:34.3750
Train: Epoch:12, Batch:79, Loss:1.903353, Acc:34.3750
Train: Epoch:12, Batch:99, Loss:1.829777, Acc:43.7500
Train: Epoch:12, Batch:119, Loss:2.002075, Acc:18.7500
Train: Epoch:12, Batch:139, Loss:1.937441, Acc:25.0000
Train: Epoch:12, Batch:159, Loss:1.982869, Acc:25.0000
Train: Epoch:12, Batch:179, Loss:1.901786, Acc:34.3750
Train: Epoch:12, Batch:199, Loss:1.955642, Acc:21.8750
Train: Epoch:12, Batch:219, Loss:2.015793, Acc:25.0000
Train: Epoch:12, Batch:239, Loss:1.895507, Acc:40.6250
Train: Epoch:12, Batch:259, Loss:1.958878, Acc:21.8750
Train: Epoch:12, Batch:279, Loss:1.958915, Acc:28.1250
Train: Epoch:12, Batch:299, Loss:1.955349, Acc:21.8750
Train: Epoch:12, Batch:319, Loss:1.917188, Acc:31.2500
Train: Epoch:12, Batch:339, Loss:1.973897, Acc:37.5000
Train: Epoch:12, Batch:359, Loss:2.010638, Acc:21.8750
Train: Epoch:12, Batch:379, Loss:1.947264, Acc:31.2500
Train: Epoch:1

Time taken = 161.1284 s

Train: Epoch:18, Batch:19, Loss:1.940713, Acc:31.2500
Train: Epoch:18, Batch:39, Loss:2.015262, Acc:28.1250
Train: Epoch:18, Batch:59, Loss:1.912801, Acc:34.3750
Train: Epoch:18, Batch:79, Loss:1.892349, Acc:34.3750
Train: Epoch:18, Batch:99, Loss:1.906319, Acc:37.5000
Train: Epoch:18, Batch:119, Loss:1.974947, Acc:31.2500
Train: Epoch:18, Batch:139, Loss:1.949012, Acc:31.2500
Train: Epoch:18, Batch:159, Loss:1.952014, Acc:28.1250
Train: Epoch:18, Batch:179, Loss:1.886081, Acc:34.3750
Train: Epoch:18, Batch:199, Loss:2.011434, Acc:21.8750
Train: Epoch:18, Batch:219, Loss:1.979301, Acc:25.0000
Train: Epoch:18, Batch:239, Loss:1.911616, Acc:37.5000
Train: Epoch:18, Batch:259, Loss:1.870131, Acc:40.6250
Train: Epoch:18, Batch:279, Loss:2.029464, Acc:21.8750
Train: Epoch:18, Batch:299, Loss:1.975598, Acc:21.8750
Train: Epoch:18, Batch:319, Loss:1.923874, Acc:34.3750
Train: Epoch:18, Batch:339, Loss:1.963507, Acc:34.3750
Train: Epoch:18, Batch:359, Loss:1.936044, Ac

Val_: Total Validation Loss: 1.996658, Acc: 25.0000
Test_: Total Testing Acc: 24.8125
Time taken = 166.3467 s

Train: Epoch:24, Batch:19, Loss:1.946685, Acc:31.2500
Train: Epoch:24, Batch:39, Loss:1.945227, Acc:31.2500
Train: Epoch:24, Batch:59, Loss:1.916852, Acc:34.3750
Train: Epoch:24, Batch:79, Loss:1.848693, Acc:50.0000
Train: Epoch:24, Batch:99, Loss:1.860502, Acc:40.6250
Train: Epoch:24, Batch:119, Loss:1.979726, Acc:28.1250
Train: Epoch:24, Batch:139, Loss:1.882809, Acc:31.2500
Train: Epoch:24, Batch:159, Loss:1.917349, Acc:31.2500
Train: Epoch:24, Batch:179, Loss:1.905153, Acc:37.5000
Train: Epoch:24, Batch:199, Loss:1.930751, Acc:25.0000
Train: Epoch:24, Batch:219, Loss:1.935878, Acc:37.5000
Train: Epoch:24, Batch:239, Loss:1.892271, Acc:34.3750
Train: Epoch:24, Batch:259, Loss:1.904468, Acc:34.3750
Train: Epoch:24, Batch:279, Loss:2.011224, Acc:18.7500
Train: Epoch:24, Batch:299, Loss:1.903114, Acc:37.5000
Train: Epoch:24, Batch:319, Loss:1.887289, Acc:37.5000
Train: Epoch:2

...

Train: Epoch:71, Batch:319, Loss:1.412163, Acc:87.5000
Train: Epoch:71, Batch:339, Loss:1.310176, Acc:96.8750
Train: Epoch:71, Batch:359, Loss:1.404926, Acc:87.5000
Train: Epoch:71, Batch:379, Loss:1.452455, Acc:81.2500
Train: Epoch:71, Batch:399, Loss:1.384222, Acc:87.5000
Train: Epoch:71, Batch:419, Loss:1.465519, Acc:81.2500
Train: Epoch:71, Batch:439, Loss:1.498984, Acc:78.1250
Train_: Avg Training Loss: 1.461154, Avg Training Acc: 81.312500
Val_: Total Validation Loss: 1.918933, Acc: 34.4625
Test_: Total Testing Acc: 34.3875
Time taken = 203.1684 s

Train: Epoch:72, Batch:19, Loss:1.524240, Acc:75.0000
Train: Epoch:72, Batch:39, Loss:1.513611, Acc:75.0000
Train: Epoch:72, Batch:59, Loss:1.521259, Acc:75.0000
Train: Epoch:72, Batch:79, Loss:1.451657, Acc:81.2500
Train: Epoch:72, Batch:99, Loss:1.496407, Acc:78.1250
Train: Epoch:72, Batch:119, Loss:1.545746, Acc:71.8750
Train: Epoch:72, Batch:139, Loss:1.504160, Acc:75.0000
Train: Epoch:72, Batch:159, Loss:1.465381, Acc:81.2500
Trai

Train: Epoch:77, Batch:299, Loss:1.446295, Acc:84.3750
Train: Epoch:77, Batch:319, Loss:1.427510, Acc:87.5000
Train: Epoch:77, Batch:339, Loss:1.350568, Acc:90.6250
Train: Epoch:77, Batch:359, Loss:1.428099, Acc:84.3750
Train: Epoch:77, Batch:379, Loss:1.492387, Acc:78.1250
Train: Epoch:77, Batch:399, Loss:1.404203, Acc:87.5000
Train: Epoch:77, Batch:419, Loss:1.536790, Acc:71.8750
Train: Epoch:77, Batch:439, Loss:1.544713, Acc:71.8750
Train_: Avg Training Loss: 1.449307, Avg Training Acc: 82.527778
Val_: Total Validation Loss: 1.921792, Acc: 34.5625
Test_: Total Testing Acc: 35.7375
Time taken = 159.3214 s

Train: Epoch:78, Batch:19, Loss:1.559459, Acc:71.8750
Train: Epoch:78, Batch:39, Loss:1.491259, Acc:78.1250
Train: Epoch:78, Batch:59, Loss:1.493683, Acc:78.1250
Train: Epoch:78, Batch:79, Loss:1.425517, Acc:84.3750
Train: Epoch:78, Batch:99, Loss:1.416444, Acc:87.5000
Train: Epoch:78, Batch:119, Loss:1.493209, Acc:78.1250
Train: Epoch:78, Batch:139, Loss:1.480340, Acc:78.1250
Trai

Train: Epoch:83, Batch:279, Loss:1.462164, Acc:81.2500
Train: Epoch:83, Batch:299, Loss:1.487532, Acc:78.1250
Train: Epoch:83, Batch:319, Loss:1.418533, Acc:84.3750
Train: Epoch:83, Batch:339, Loss:1.307585, Acc:96.8750
Train: Epoch:83, Batch:359, Loss:1.368220, Acc:90.6250
Train: Epoch:83, Batch:379, Loss:1.462580, Acc:81.2500
Train: Epoch:83, Batch:399, Loss:1.438429, Acc:81.2500
Train: Epoch:83, Batch:419, Loss:1.492506, Acc:78.1250
Train: Epoch:83, Batch:439, Loss:1.466424, Acc:81.2500
Train_: Avg Training Loss: 1.435506, Avg Training Acc: 83.895833
Val_: Total Validation Loss: 1.923588, Acc: 34.3000
Test_: Total Testing Acc: 34.8125
Time taken = 166.4959 s

Train: Epoch:84, Batch:19, Loss:1.524571, Acc:75.0000
Train: Epoch:84, Batch:39, Loss:1.536813, Acc:71.8750
Train: Epoch:84, Batch:59, Loss:1.493069, Acc:78.1250
Train: Epoch:84, Batch:79, Loss:1.400212, Acc:87.5000
Train: Epoch:84, Batch:99, Loss:1.386043, Acc:87.5000
Train: Epoch:84, Batch:119, Loss:1.521274, Acc:75.0000
Trai

Train: Epoch:89, Batch:259, Loss:1.463535, Acc:81.2500
Train: Epoch:89, Batch:279, Loss:1.448986, Acc:81.2500
Train: Epoch:89, Batch:299, Loss:1.460630, Acc:81.2500
Train: Epoch:89, Batch:319, Loss:1.346215, Acc:93.7500
Train: Epoch:89, Batch:339, Loss:1.308070, Acc:96.8750
Train: Epoch:89, Batch:359, Loss:1.444239, Acc:84.3750
Train: Epoch:89, Batch:379, Loss:1.431756, Acc:84.3750
Train: Epoch:89, Batch:399, Loss:1.372716, Acc:90.6250
Train: Epoch:89, Batch:419, Loss:1.476871, Acc:81.2500
Train: Epoch:89, Batch:439, Loss:1.459509, Acc:81.2500
Train_: Avg Training Loss: 1.427885, Avg Training Acc: 84.638889
Val_: Total Validation Loss: 1.915547, Acc: 35.1000
Test_: Total Testing Acc: 35.5000
Time taken = 168.5414 s

Train: Epoch:90, Batch:19, Loss:1.542643, Acc:71.8750
Train: Epoch:90, Batch:39, Loss:1.493580, Acc:75.0000
Train: Epoch:90, Batch:59, Loss:1.524421, Acc:75.0000
Train: Epoch:90, Batch:79, Loss:1.430312, Acc:84.3750
Train: Epoch:90, Batch:99, Loss:1.369664, Acc:90.6250
Trai

Train: Epoch:95, Batch:239, Loss:1.368114, Acc:90.6250
Train: Epoch:95, Batch:259, Loss:1.528221, Acc:75.0000
Train: Epoch:95, Batch:279, Loss:1.490506, Acc:78.1250
Train: Epoch:95, Batch:299, Loss:1.442565, Acc:81.2500
Train: Epoch:95, Batch:319, Loss:1.399242, Acc:87.5000
Train: Epoch:95, Batch:339, Loss:1.305498, Acc:96.8750
Train: Epoch:95, Batch:359, Loss:1.423934, Acc:87.5000
Train: Epoch:95, Batch:379, Loss:1.412351, Acc:87.5000
Train: Epoch:95, Batch:399, Loss:1.379906, Acc:90.6250
Train: Epoch:95, Batch:419, Loss:1.474249, Acc:78.1250
Train: Epoch:95, Batch:439, Loss:1.495682, Acc:78.1250
Train_: Avg Training Loss: 1.422984, Avg Training Acc: 85.083333
Val_: Total Validation Loss: 1.912570, Acc: 35.4250
Test_: Total Testing Acc: 35.6875
Time taken = 212.0474 s

Train: Epoch:96, Batch:19, Loss:1.461148, Acc:81.2500
Train: Epoch:96, Batch:39, Loss:1.489235, Acc:78.1250
Train: Epoch:96, Batch:59, Loss:1.495590, Acc:78.1250
Train: Epoch:96, Batch:79, Loss:1.398437, Acc:87.5000
Tra

In [ ]:
#Re-running the model 80% data
SAVE_FILE = "new_resnet_80dat_100eps" + time.strftime("%Y-%m-%d_%H:%M:%S", time.gmtime()) + "_" + str(args.perc_train)

for epoch in range(0, args.epochs):
    t0 = time.time()
    train(epoch, SAVE_FILE)
    avg_loss, avg_acc = validate(epoch, SAVE_FILE)
    test_acc = test(epoch, SAVE_FILE)
    model.save_model(args.save, epoch, avg_acc, avg_loss)
    print("Time taken = {:.4f} s\n".format(time.time() - t0))

Train: Epoch:0, Batch:19, Loss:2.082544, Acc:9.3750
Train: Epoch:0, Batch:39, Loss:2.080004, Acc:12.5000
Train: Epoch:0, Batch:59, Loss:2.079978, Acc:12.5000
Train: Epoch:0, Batch:79, Loss:2.080541, Acc:15.6250
Train: Epoch:0, Batch:99, Loss:2.078685, Acc:12.5000
Train: Epoch:0, Batch:119, Loss:2.071007, Acc:21.8750
Train: Epoch:0, Batch:139, Loss:2.071787, Acc:12.5000
Train: Epoch:0, Batch:159, Loss:2.078464, Acc:15.6250
Train: Epoch:0, Batch:179, Loss:2.097376, Acc:12.5000
Train: Epoch:0, Batch:199, Loss:2.091815, Acc:18.7500
Train: Epoch:0, Batch:219, Loss:2.069453, Acc:12.5000
Train: Epoch:0, Batch:239, Loss:2.084078, Acc:12.5000
Train: Epoch:0, Batch:259, Loss:2.084878, Acc:9.3750
Train: Epoch:0, Batch:279, Loss:2.054013, Acc:12.5000
Train: Epoch:0, Batch:299, Loss:2.050910, Acc:15.6250
Train: Epoch:0, Batch:319, Loss:2.074566, Acc:12.5000
Train: Epoch:0, Batch:339, Loss:2.060752, Acc:15.6250
Train: Epoch:0, Batch:359, Loss:2.019306, Acc:18.7500
Train: Epoch:0, Batch:379, Loss:2.0

Train: Epoch:4, Batch:419, Loss:2.081248, Acc:9.3750
Train: Epoch:4, Batch:439, Loss:2.033590, Acc:18.7500
Train: Epoch:4, Batch:459, Loss:1.980380, Acc:28.1250
Train: Epoch:4, Batch:479, Loss:2.037903, Acc:21.8750
Train: Epoch:4, Batch:499, Loss:1.948690, Acc:25.0000
Train: Epoch:4, Batch:519, Loss:2.063761, Acc:12.5000
Train: Epoch:4, Batch:539, Loss:2.059175, Acc:21.8750
Train: Epoch:4, Batch:559, Loss:1.974525, Acc:18.7500
Train: Epoch:4, Batch:579, Loss:1.980465, Acc:21.8750
Train: Epoch:4, Batch:599, Loss:2.041247, Acc:15.6250
Train_: Avg Training Loss: 2.020218, Avg Training Acc: 20.364583
Val_: Total Validation Loss: 2.025925, Acc: 19.8750
Test_: Total Testing Acc: 20.6000
Time taken = 225.1772 s

Train: Epoch:5, Batch:19, Loss:1.980782, Acc:21.8750
Train: Epoch:5, Batch:39, Loss:1.988246, Acc:18.7500
Train: Epoch:5, Batch:59, Loss:2.027384, Acc:21.8750
Train: Epoch:5, Batch:79, Loss:1.996069, Acc:21.8750
Train: Epoch:5, Batch:99, Loss:2.015696, Acc:18.7500
Train: Epoch:5, Batc

Train: Epoch:9, Batch:159, Loss:1.996808, Acc:15.6250
Train: Epoch:9, Batch:179, Loss:2.057651, Acc:21.8750
Train: Epoch:9, Batch:199, Loss:2.066906, Acc:15.6250
Train: Epoch:9, Batch:219, Loss:2.030997, Acc:15.6250
Train: Epoch:9, Batch:239, Loss:1.969572, Acc:34.3750
Train: Epoch:9, Batch:259, Loss:2.034078, Acc:15.6250
Train: Epoch:9, Batch:279, Loss:1.989587, Acc:18.7500
Train: Epoch:9, Batch:299, Loss:1.978182, Acc:37.5000
Train: Epoch:9, Batch:319, Loss:2.023735, Acc:25.0000
Train: Epoch:9, Batch:339, Loss:2.027455, Acc:21.8750
Train: Epoch:9, Batch:359, Loss:2.042915, Acc:18.7500
Train: Epoch:9, Batch:379, Loss:1.962114, Acc:34.3750
Train: Epoch:9, Batch:399, Loss:2.074286, Acc:12.5000
Train: Epoch:9, Batch:419, Loss:2.062074, Acc:9.3750
Train: Epoch:9, Batch:439, Loss:2.100159, Acc:15.6250
Train: Epoch:9, Batch:459, Loss:1.957253, Acc:25.0000
Train: Epoch:9, Batch:479, Loss:2.036695, Acc:21.8750
Train: Epoch:9, Batch:499, Loss:1.947801, Acc:34.3750
Train: Epoch:9, Batch:519, Lo

Train: Epoch:13, Batch:519, Loss:2.037724, Acc:18.7500
Train: Epoch:13, Batch:539, Loss:2.044531, Acc:18.7500
Train: Epoch:13, Batch:559, Loss:1.920047, Acc:31.2500
Train: Epoch:13, Batch:579, Loss:1.967808, Acc:25.0000
Train: Epoch:13, Batch:599, Loss:1.902726, Acc:46.8750
Train_: Avg Training Loss: 1.984208, Avg Training Acc: 25.812500
Val_: Total Validation Loss: 1.983661, Acc: 25.8375
Test_: Total Testing Acc: 26.5625
Time taken = 198.1806 s

Train: Epoch:14, Batch:19, Loss:1.934541, Acc:28.1250
Train: Epoch:14, Batch:39, Loss:1.996786, Acc:28.1250
Train: Epoch:14, Batch:59, Loss:1.997077, Acc:21.8750
Train: Epoch:14, Batch:79, Loss:1.951602, Acc:31.2500
Train: Epoch:14, Batch:99, Loss:1.979751, Acc:25.0000
Train: Epoch:14, Batch:119, Loss:1.923853, Acc:37.5000
Train: Epoch:14, Batch:139, Loss:1.924025, Acc:37.5000
Train: Epoch:14, Batch:159, Loss:1.990645, Acc:15.6250
Train: Epoch:14, Batch:179, Loss:2.036992, Acc:21.8750
Train: Epoch:14, Batch:199, Loss:2.001606, Acc:18.7500
Trai

Train: Epoch:18, Batch:199, Loss:1.971664, Acc:28.1250
Train: Epoch:18, Batch:219, Loss:1.951443, Acc:25.0000
Train: Epoch:18, Batch:239, Loss:1.975082, Acc:25.0000
Train: Epoch:18, Batch:259, Loss:2.014733, Acc:21.8750
Train: Epoch:18, Batch:279, Loss:1.836890, Acc:40.6250
Train: Epoch:18, Batch:299, Loss:1.962985, Acc:28.1250
Train: Epoch:18, Batch:319, Loss:1.993795, Acc:28.1250
Train: Epoch:18, Batch:339, Loss:1.956794, Acc:31.2500
Train: Epoch:18, Batch:359, Loss:1.957503, Acc:28.1250
Train: Epoch:18, Batch:379, Loss:1.953200, Acc:34.3750
Train: Epoch:18, Batch:399, Loss:2.020782, Acc:25.0000
Train: Epoch:18, Batch:419, Loss:1.989172, Acc:25.0000
Train: Epoch:18, Batch:439, Loss:1.947664, Acc:31.2500
Train: Epoch:18, Batch:459, Loss:1.930712, Acc:34.3750
Train: Epoch:18, Batch:479, Loss:1.908580, Acc:31.2500
Train: Epoch:18, Batch:499, Loss:1.926631, Acc:31.2500
Train: Epoch:18, Batch:519, Loss:1.967714, Acc:28.1250
Train: Epoch:18, Batch:539, Loss:1.971144, Acc:34.3750
Train: Epo

...

Train: Epoch:81, Batch:339, Loss:1.467820, Acc:81.2500
Train: Epoch:81, Batch:359, Loss:1.376329, Acc:90.6250
Train: Epoch:81, Batch:379, Loss:1.428628, Acc:84.3750
Train: Epoch:81, Batch:399, Loss:1.433327, Acc:84.3750
Train: Epoch:81, Batch:419, Loss:1.458677, Acc:81.2500
Train: Epoch:81, Batch:439, Loss:1.375270, Acc:90.6250
Train: Epoch:81, Batch:459, Loss:1.477006, Acc:78.1250
Train: Epoch:81, Batch:479, Loss:1.392808, Acc:87.5000
Train: Epoch:81, Batch:499, Loss:1.473933, Acc:78.1250
Train: Epoch:81, Batch:519, Loss:1.521008, Acc:75.0000
Train: Epoch:81, Batch:539, Loss:1.411510, Acc:87.5000
Train: Epoch:81, Batch:559, Loss:1.398890, Acc:87.5000
Train: Epoch:81, Batch:579, Loss:1.430375, Acc:84.3750
Train: Epoch:81, Batch:599, Loss:1.395041, Acc:87.5000
Train_: Avg Training Loss: 1.444726, Avg Training Acc: 82.901042
Val_: Total Validation Loss: 1.890228, Acc: 37.7125
Test_: Total Testing Acc: 37.4750
Time taken = 194.3404 s

Train: Epoch:82, Batch:19, Loss:1.471688, Acc:81.2500


Time taken = 192.4445 s

Train: Epoch:86, Batch:19, Loss:1.465657, Acc:81.2500
Train: Epoch:86, Batch:39, Loss:1.477637, Acc:78.1250
Train: Epoch:86, Batch:59, Loss:1.344278, Acc:93.7500
Train: Epoch:86, Batch:79, Loss:1.397538, Acc:84.3750
Train: Epoch:86, Batch:99, Loss:1.408327, Acc:87.5000
Train: Epoch:86, Batch:119, Loss:1.521991, Acc:75.0000
Train: Epoch:86, Batch:139, Loss:1.376165, Acc:90.6250
Train: Epoch:86, Batch:159, Loss:1.401315, Acc:87.5000
Train: Epoch:86, Batch:179, Loss:1.419500, Acc:84.3750
Train: Epoch:86, Batch:199, Loss:1.398593, Acc:87.5000
Train: Epoch:86, Batch:219, Loss:1.536521, Acc:75.0000
Train: Epoch:86, Batch:239, Loss:1.444815, Acc:84.3750
Train: Epoch:86, Batch:259, Loss:1.399394, Acc:87.5000
Train: Epoch:86, Batch:279, Loss:1.392099, Acc:87.5000
Train: Epoch:86, Batch:299, Loss:1.493670, Acc:78.1250
Train: Epoch:86, Batch:319, Loss:1.550944, Acc:71.8750
Train: Epoch:86, Batch:339, Loss:1.423935, Acc:84.3750
Train: Epoch:86, Batch:359, Loss:1.433962, Ac

Train: Epoch:90, Batch:359, Loss:1.385100, Acc:90.6250
Train: Epoch:90, Batch:379, Loss:1.430359, Acc:84.3750
Train: Epoch:90, Batch:399, Loss:1.436364, Acc:84.3750
Train: Epoch:90, Batch:419, Loss:1.432029, Acc:84.3750
Train: Epoch:90, Batch:439, Loss:1.398506, Acc:87.5000
Train: Epoch:90, Batch:459, Loss:1.479778, Acc:81.2500
Train: Epoch:90, Batch:479, Loss:1.387784, Acc:90.6250
Train: Epoch:90, Batch:499, Loss:1.462037, Acc:81.2500
Train: Epoch:90, Batch:519, Loss:1.456177, Acc:81.2500
Train: Epoch:90, Batch:539, Loss:1.408309, Acc:84.3750
Train: Epoch:90, Batch:559, Loss:1.397543, Acc:87.5000
Train: Epoch:90, Batch:579, Loss:1.440825, Acc:84.3750
Train: Epoch:90, Batch:599, Loss:1.381529, Acc:90.6250
Train_: Avg Training Loss: 1.424135, Avg Training Acc: 85.036458
Val_: Total Validation Loss: 1.874646, Acc: 39.3750
Test_: Total Testing Acc: 39.2250
Time taken = 190.3962 s

Train: Epoch:91, Batch:19, Loss:1.571338, Acc:71.8750
Train: Epoch:91, Batch:39, Loss:1.434161, Acc:84.3750
T

Train: Epoch:95, Batch:39, Loss:1.377561, Acc:90.6250
Train: Epoch:95, Batch:59, Loss:1.314351, Acc:96.8750
Train: Epoch:95, Batch:79, Loss:1.362141, Acc:90.6250
Train: Epoch:95, Batch:99, Loss:1.394374, Acc:87.5000
Train: Epoch:95, Batch:119, Loss:1.461603, Acc:81.2500
Train: Epoch:95, Batch:139, Loss:1.395581, Acc:87.5000
Train: Epoch:95, Batch:159, Loss:1.463183, Acc:81.2500
Train: Epoch:95, Batch:179, Loss:1.404096, Acc:87.5000
Train: Epoch:95, Batch:199, Loss:1.414942, Acc:87.5000
Train: Epoch:95, Batch:219, Loss:1.568716, Acc:71.8750
Train: Epoch:95, Batch:239, Loss:1.431677, Acc:84.3750
Train: Epoch:95, Batch:259, Loss:1.397172, Acc:87.5000
Train: Epoch:95, Batch:279, Loss:1.353866, Acc:93.7500
Train: Epoch:95, Batch:299, Loss:1.476950, Acc:81.2500
Train: Epoch:95, Batch:319, Loss:1.462109, Acc:81.2500
Train: Epoch:95, Batch:339, Loss:1.496541, Acc:78.1250
Train: Epoch:95, Batch:359, Loss:1.395945, Acc:87.5000
Train: Epoch:95, Batch:379, Loss:1.430234, Acc:87.5000
Train: Epoch:9

Train: Epoch:99, Batch:379, Loss:1.485224, Acc:78.1250
Train: Epoch:99, Batch:399, Loss:1.537769, Acc:71.8750
Train: Epoch:99, Batch:419, Loss:1.515181, Acc:75.0000
Train: Epoch:99, Batch:439, Loss:1.370960, Acc:90.6250
Train: Epoch:99, Batch:459, Loss:1.467272, Acc:78.1250
Train: Epoch:99, Batch:479, Loss:1.423230, Acc:84.3750
Train: Epoch:99, Batch:499, Loss:1.420166, Acc:84.3750
Train: Epoch:99, Batch:519, Loss:1.491026, Acc:78.1250
Train: Epoch:99, Batch:539, Loss:1.388570, Acc:87.5000
Train: Epoch:99, Batch:559, Loss:1.399501, Acc:87.5000
Train: Epoch:99, Batch:579, Loss:1.497347, Acc:78.1250
Train: Epoch:99, Batch:599, Loss:1.431901, Acc:84.3750
Train_: Avg Training Loss: 1.413968, Avg Training Acc: 86.000000
Val_: Total Validation Loss: 1.863612, Acc: 40.6750
Test_: Total Testing Acc: 39.9500
Time taken = 230.8439 s



In [ ]:
#Re-running the model 40% data
SAVE_FILE = "new_resnet_40dat_100eps" + time.strftime("%Y-%m-%d_%H:%M:%S", time.gmtime()) + "_" + str(args.perc_train)

for epoch in range(0, args.epochs):
    t0 = time.time()
    train(epoch, SAVE_FILE)
    avg_loss, avg_acc = validate(epoch, SAVE_FILE)
    test_acc = test(epoch, SAVE_FILE)
    model.save_model(args.save, epoch, avg_acc, avg_loss)
    print("Time taken = {:.4f} s\n".format(time.time() - t0))

Train: Epoch:0, Batch:19, Loss:2.080605, Acc:15.6250
Train: Epoch:0, Batch:39, Loss:2.077346, Acc:15.6250
Train: Epoch:0, Batch:59, Loss:2.080094, Acc:12.5000
Train: Epoch:0, Batch:79, Loss:2.088710, Acc:9.3750
Train: Epoch:0, Batch:99, Loss:2.083915, Acc:3.1250
Train: Epoch:0, Batch:119, Loss:2.079226, Acc:12.5000
Train: Epoch:0, Batch:139, Loss:2.081377, Acc:9.3750
Train: Epoch:0, Batch:159, Loss:2.081397, Acc:9.3750
Train: Epoch:0, Batch:179, Loss:2.087690, Acc:3.1250
Train: Epoch:0, Batch:199, Loss:2.063433, Acc:18.7500
Train: Epoch:0, Batch:219, Loss:2.073493, Acc:18.7500
Train: Epoch:0, Batch:239, Loss:2.080116, Acc:6.2500
Train: Epoch:0, Batch:259, Loss:2.099381, Acc:9.3750
Train: Epoch:0, Batch:279, Loss:2.080319, Acc:18.7500
Train: Epoch:0, Batch:299, Loss:2.026987, Acc:25.0000
Train_: Avg Training Loss: 2.076925, Avg Training Acc: 14.291667
Val_: Total Validation Loss: 2.087205, Acc: 14.8750
Test_: Total Testing Acc: 14.9000
Time taken = 167.5672 s

Train: Epoch:1, Batch:19, 

Train: Epoch:8, Batch:159, Loss:2.004659, Acc:18.7500
Train: Epoch:8, Batch:179, Loss:2.118399, Acc:3.1250
Train: Epoch:8, Batch:199, Loss:2.059536, Acc:21.8750
Train: Epoch:8, Batch:219, Loss:2.054062, Acc:18.7500
Train: Epoch:8, Batch:239, Loss:2.007831, Acc:25.0000
Train: Epoch:8, Batch:259, Loss:2.076015, Acc:15.6250
Train: Epoch:8, Batch:279, Loss:2.041123, Acc:25.0000
Train: Epoch:8, Batch:299, Loss:1.928723, Acc:31.2500
Train_: Avg Training Loss: 2.015427, Avg Training Acc: 22.031250
Val_: Total Validation Loss: 2.024085, Acc: 20.2750
Test_: Total Testing Acc: 21.0500
Time taken = 168.3581 s

Train: Epoch:9, Batch:19, Loss:2.033667, Acc:18.7500
Train: Epoch:9, Batch:39, Loss:1.965821, Acc:21.8750
Train: Epoch:9, Batch:59, Loss:1.966863, Acc:25.0000
Train: Epoch:9, Batch:79, Loss:1.929488, Acc:34.3750
Train: Epoch:9, Batch:99, Loss:2.006661, Acc:25.0000
Train: Epoch:9, Batch:119, Loss:2.016911, Acc:25.0000
Train: Epoch:9, Batch:139, Loss:1.955263, Acc:25.0000
Train: Epoch:9, Batc

Train: Epoch:16, Batch:259, Loss:2.051879, Acc:18.7500
Train: Epoch:16, Batch:279, Loss:1.889166, Acc:37.5000
Train: Epoch:16, Batch:299, Loss:1.969622, Acc:25.0000
Train_: Avg Training Loss: 1.937314, Avg Training Acc: 32.479167
Val_: Total Validation Loss: 1.995847, Acc: 26.6000
Test_: Total Testing Acc: 26.5625
Time taken = 161.1094 s

Train: Epoch:17, Batch:19, Loss:1.877705, Acc:34.3750
Train: Epoch:17, Batch:39, Loss:1.890326, Acc:34.3750
Train: Epoch:17, Batch:59, Loss:1.888335, Acc:40.6250
Train: Epoch:17, Batch:79, Loss:1.853017, Acc:37.5000
Train: Epoch:17, Batch:99, Loss:1.882618, Acc:43.7500
Train: Epoch:17, Batch:119, Loss:1.959329, Acc:25.0000
Train: Epoch:17, Batch:139, Loss:1.900158, Acc:37.5000
Train: Epoch:17, Batch:159, Loss:1.911638, Acc:31.2500
Train: Epoch:17, Batch:179, Loss:2.035491, Acc:18.7500
Train: Epoch:17, Batch:199, Loss:1.975818, Acc:28.1250
Train: Epoch:17, Batch:219, Loss:1.914061, Acc:34.3750
Train: Epoch:17, Batch:239, Loss:1.954287, Acc:28.1250
Trai

Val_: Total Validation Loss: 1.983889, Acc: 27.8000
Test_: Total Testing Acc: 26.9625
Time taken = 190.7493 s

Train: Epoch:25, Batch:19, Loss:1.657445, Acc:65.6250
Train: Epoch:25, Batch:39, Loss:1.753246, Acc:53.1250
Train: Epoch:25, Batch:59, Loss:1.746168, Acc:53.1250
Train: Epoch:25, Batch:79, Loss:1.755898, Acc:50.0000
Train: Epoch:25, Batch:99, Loss:1.782559, Acc:46.8750
Train: Epoch:25, Batch:119, Loss:1.805843, Acc:53.1250
Train: Epoch:25, Batch:139, Loss:1.759171, Acc:50.0000
Train: Epoch:25, Batch:159, Loss:1.764624, Acc:46.8750
Train: Epoch:25, Batch:179, Loss:1.869238, Acc:40.6250
Train: Epoch:25, Batch:199, Loss:1.817249, Acc:40.6250
Train: Epoch:25, Batch:219, Loss:1.700815, Acc:56.2500
Train: Epoch:25, Batch:239, Loss:1.860178, Acc:43.7500
Train: Epoch:25, Batch:259, Loss:1.890845, Acc:40.6250
Train: Epoch:25, Batch:279, Loss:1.715221, Acc:59.3750
Train: Epoch:25, Batch:299, Loss:1.667181, Acc:59.3750
Train_: Avg Training Loss: 1.781672, Avg Training Acc: 48.916667
Val_

Train: Epoch:33, Batch:79, Loss:1.696717, Acc:56.2500
Train: Epoch:33, Batch:99, Loss:1.631064, Acc:65.6250
Train: Epoch:33, Batch:119, Loss:1.716257, Acc:56.2500
Train: Epoch:33, Batch:139, Loss:1.697352, Acc:56.2500
Train: Epoch:33, Batch:159, Loss:1.670148, Acc:59.3750
Train: Epoch:33, Batch:179, Loss:1.725708, Acc:59.3750
Train: Epoch:33, Batch:199, Loss:1.706358, Acc:56.2500
Train: Epoch:33, Batch:219, Loss:1.620032, Acc:65.6250
Train: Epoch:33, Batch:239, Loss:1.727138, Acc:56.2500
Train: Epoch:33, Batch:259, Loss:1.832821, Acc:40.6250
Train: Epoch:33, Batch:279, Loss:1.685584, Acc:56.2500
Train: Epoch:33, Batch:299, Loss:1.528219, Acc:75.0000
Train_: Avg Training Loss: 1.687042, Avg Training Acc: 58.125000
Val_: Total Validation Loss: 1.965042, Acc: 29.4875
Test_: Total Testing Acc: 29.4000
Time taken = 163.2862 s

Train: Epoch:34, Batch:19, Loss:1.571828, Acc:68.7500
Train: Epoch:34, Batch:39, Loss:1.701743, Acc:56.2500
Train: Epoch:34, Batch:59, Loss:1.715690, Acc:53.1250
Trai

...

Train: Epoch:49, Batch:279, Loss:1.573223, Acc:71.8750
Train: Epoch:49, Batch:299, Loss:1.446738, Acc:84.3750
Train_: Avg Training Loss: 1.528675, Avg Training Acc: 74.750000
Val_: Total Validation Loss: 1.979345, Acc: 28.5250
Test_: Total Testing Acc: 28.6750
Time taken = 170.5721 s

Train: Epoch:50, Batch:19, Loss:1.424223, Acc:87.5000
Train: Epoch:50, Batch:39, Loss:1.435749, Acc:84.3750
Train: Epoch:50, Batch:59, Loss:1.579285, Acc:68.7500
Train: Epoch:50, Batch:79, Loss:1.634526, Acc:62.5000
Train: Epoch:50, Batch:99, Loss:1.471777, Acc:81.2500
Train: Epoch:50, Batch:119, Loss:1.572319, Acc:71.8750
Train: Epoch:50, Batch:139, Loss:1.404539, Acc:84.3750
Train: Epoch:50, Batch:159, Loss:1.403750, Acc:87.5000
Train: Epoch:50, Batch:179, Loss:1.606709, Acc:68.7500
Train: Epoch:50, Batch:199, Loss:1.488421, Acc:81.2500
Train: Epoch:50, Batch:219, Loss:1.500133, Acc:78.1250
Train: Epoch:50, Batch:239, Loss:1.573781, Acc:68.7500
Train: Epoch:50, Batch:259, Loss:1.663704, Acc:59.3750
Trai

Test_: Total Testing Acc: 27.6750
Time taken = 165.1399 s

Train: Epoch:58, Batch:19, Loss:1.291560, Acc:100.0000
Train: Epoch:58, Batch:39, Loss:1.465320, Acc:81.2500
Train: Epoch:58, Batch:59, Loss:1.477753, Acc:81.2500
Train: Epoch:58, Batch:79, Loss:1.547374, Acc:75.0000
Train: Epoch:58, Batch:99, Loss:1.379885, Acc:90.6250
Train: Epoch:58, Batch:119, Loss:1.439317, Acc:84.3750
Train: Epoch:58, Batch:139, Loss:1.422348, Acc:84.3750
Train: Epoch:58, Batch:159, Loss:1.408425, Acc:87.5000
Train: Epoch:58, Batch:179, Loss:1.614285, Acc:65.6250
Train: Epoch:58, Batch:199, Loss:1.464632, Acc:81.2500
Train: Epoch:58, Batch:219, Loss:1.480475, Acc:81.2500
Train: Epoch:58, Batch:239, Loss:1.456494, Acc:81.2500
Train: Epoch:58, Batch:259, Loss:1.579587, Acc:71.8750
Train: Epoch:58, Batch:279, Loss:1.502065, Acc:78.1250
Train: Epoch:58, Batch:299, Loss:1.377738, Acc:90.6250
Train_: Avg Training Loss: 1.482194, Avg Training Acc: 79.312500
Val_: Total Validation Loss: 1.978919, Acc: 28.6625
Tes

Train: Epoch:66, Batch:99, Loss:1.368377, Acc:90.6250
Train: Epoch:66, Batch:119, Loss:1.358880, Acc:93.7500
Train: Epoch:66, Batch:139, Loss:1.334676, Acc:93.7500
Train: Epoch:66, Batch:159, Loss:1.426080, Acc:87.5000
Train: Epoch:66, Batch:179, Loss:1.490658, Acc:78.1250
Train: Epoch:66, Batch:199, Loss:1.433050, Acc:84.3750
Train: Epoch:66, Batch:219, Loss:1.500245, Acc:78.1250
Train: Epoch:66, Batch:239, Loss:1.359596, Acc:93.7500
Train: Epoch:66, Batch:259, Loss:1.460001, Acc:81.2500
Train: Epoch:66, Batch:279, Loss:1.475695, Acc:78.1250
Train: Epoch:66, Batch:299, Loss:1.372845, Acc:90.6250
Train_: Avg Training Loss: 1.448426, Avg Training Acc: 82.750000
Val_: Total Validation Loss: 1.987682, Acc: 27.7625
Test_: Total Testing Acc: 27.5750
Time taken = 166.9098 s

Train: Epoch:67, Batch:19, Loss:1.365950, Acc:87.5000
Train: Epoch:67, Batch:39, Loss:1.421943, Acc:84.3750
Train: Epoch:67, Batch:59, Loss:1.502849, Acc:78.1250
Train: Epoch:67, Batch:79, Loss:1.474984, Acc:78.1250
Trai

Train: Epoch:74, Batch:199, Loss:1.473572, Acc:81.2500
Train: Epoch:74, Batch:219, Loss:1.431440, Acc:84.3750
Train: Epoch:74, Batch:239, Loss:1.397396, Acc:87.5000
Train: Epoch:74, Batch:259, Loss:1.511683, Acc:75.0000
Train: Epoch:74, Batch:279, Loss:1.526128, Acc:75.0000
Train: Epoch:74, Batch:299, Loss:1.395524, Acc:87.5000
Train_: Avg Training Loss: 1.438500, Avg Training Acc: 83.708333
Val_: Total Validation Loss: 1.981909, Acc: 28.4500
Test_: Total Testing Acc: 28.9000
Time taken = 144.4659 s

Train: Epoch:75, Batch:19, Loss:1.353452, Acc:90.6250
Train: Epoch:75, Batch:39, Loss:1.411676, Acc:87.5000
Train: Epoch:75, Batch:59, Loss:1.380378, Acc:90.6250
Train: Epoch:75, Batch:79, Loss:1.483261, Acc:78.1250
Train: Epoch:75, Batch:99, Loss:1.361511, Acc:90.6250
Train: Epoch:75, Batch:119, Loss:1.389780, Acc:87.5000
Train: Epoch:75, Batch:139, Loss:1.358966, Acc:90.6250
Train: Epoch:75, Batch:159, Loss:1.420085, Acc:84.3750
Train: Epoch:75, Batch:179, Loss:1.516585, Acc:75.0000
Trai

Train: Epoch:82, Batch:299, Loss:1.371270, Acc:90.6250
Train_: Avg Training Loss: 1.424624, Avg Training Acc: 85.031250


In [ ]:
#Re-running the model 20% data
SAVE_FILE = "new_resnet_20dat_100eps" + time.strftime("%Y-%m-%d_%H:%M:%S", time.gmtime()) + "_" + str(args.perc_train)

for epoch in range(0, args.epochs):
    t0 = time.time()
    train(epoch, SAVE_FILE)
    avg_loss, avg_acc = validate(epoch, SAVE_FILE)
    test_acc = test(epoch, SAVE_FILE)
    model.save_model(args.save, epoch, avg_acc, avg_loss)
    print("Time taken = {:.4f} s\n".format(time.time() - t0))

Train: Epoch:0, Batch:19, Loss:2.073840, Acc:18.7500
Train: Epoch:0, Batch:39, Loss:2.080431, Acc:18.7500
Train: Epoch:0, Batch:59, Loss:2.075001, Acc:18.7500
Train: Epoch:0, Batch:79, Loss:2.076471, Acc:18.7500
Train: Epoch:0, Batch:99, Loss:2.078951, Acc:15.6250
Train: Epoch:0, Batch:119, Loss:2.078081, Acc:18.7500
Train: Epoch:0, Batch:139, Loss:2.091138, Acc:3.1250
Train_: Avg Training Loss: 2.079210, Avg Training Acc: 12.958333
Val_: Total Validation Loss: 2.078378, Acc: 15.1000
Test_: Total Testing Acc: 14.2875
Time taken = 119.8147 s

Train: Epoch:1, Batch:19, Loss:2.057613, Acc:31.2500
Train: Epoch:1, Batch:39, Loss:2.064300, Acc:12.5000
Train: Epoch:1, Batch:59, Loss:2.098686, Acc:6.2500
Train: Epoch:1, Batch:79, Loss:2.002530, Acc:31.2500
Train: Epoch:1, Batch:99, Loss:1.990686, Acc:25.0000
Train: Epoch:1, Batch:119, Loss:2.063397, Acc:12.5000
Train: Epoch:1, Batch:139, Loss:2.066690, Acc:18.7500
Train_: Avg Training Loss: 2.059455, Avg Training Acc: 17.625000
Val_: Total Val

Test_: Total Testing Acc: 21.6250
Time taken = 117.6904 s

Train: Epoch:15, Batch:19, Loss:1.810127, Acc:46.8750
Train: Epoch:15, Batch:39, Loss:1.998291, Acc:21.8750
Train: Epoch:15, Batch:59, Loss:1.941149, Acc:25.0000
Train: Epoch:15, Batch:79, Loss:1.805340, Acc:43.7500
Train: Epoch:15, Batch:99, Loss:1.822771, Acc:43.7500
Train: Epoch:15, Batch:119, Loss:2.035158, Acc:21.8750
Train: Epoch:15, Batch:139, Loss:1.965158, Acc:28.1250
Train_: Avg Training Loss: 1.934113, Avg Training Acc: 32.062500
Val_: Total Validation Loss: 2.017242, Acc: 23.0125
Test_: Total Testing Acc: 22.5375
Time taken = 117.1784 s

Train: Epoch:16, Batch:19, Loss:1.767717, Acc:53.1250
Train: Epoch:16, Batch:39, Loss:1.889112, Acc:37.5000
Train: Epoch:16, Batch:59, Loss:1.926315, Acc:31.2500
Train: Epoch:16, Batch:79, Loss:1.822901, Acc:46.8750
Train: Epoch:16, Batch:99, Loss:1.814905, Acc:43.7500
Train: Epoch:16, Batch:119, Loss:1.976902, Acc:28.1250
Train: Epoch:16, Batch:139, Loss:2.014954, Acc:28.1250
Train

Train_: Avg Training Loss: 1.604709, Avg Training Acc: 67.583333
Val_: Total Validation Loss: 2.057586, Acc: 19.4500
Test_: Total Testing Acc: 20.3500
Time taken = 109.5137 s

Train: Epoch:30, Batch:19, Loss:1.580567, Acc:71.8750
Train: Epoch:30, Batch:39, Loss:1.664602, Acc:59.3750
Train: Epoch:30, Batch:59, Loss:1.506024, Acc:78.1250
Train: Epoch:30, Batch:79, Loss:1.439239, Acc:84.3750
Train: Epoch:30, Batch:99, Loss:1.663315, Acc:59.3750
Train: Epoch:30, Batch:119, Loss:1.583311, Acc:68.7500
Train: Epoch:30, Batch:139, Loss:1.514320, Acc:78.1250
Train_: Avg Training Loss: 1.586007, Avg Training Acc: 69.562500
Val_: Total Validation Loss: 2.050292, Acc: 20.6125
Test_: Total Testing Acc: 20.4000
Time taken = 108.3736 s

Train: Epoch:31, Batch:19, Loss:1.591752, Acc:68.7500
Train: Epoch:31, Batch:39, Loss:1.650695, Acc:62.5000
Train: Epoch:31, Batch:59, Loss:1.546340, Acc:75.0000
Train: Epoch:31, Batch:79, Loss:1.461936, Acc:81.2500
Train: Epoch:31, Batch:99, Loss:1.576347, Acc:71.875

Train: Epoch:44, Batch:119, Loss:1.381093, Acc:90.6250
Train: Epoch:44, Batch:139, Loss:1.460128, Acc:81.2500
Train_: Avg Training Loss: 1.450528, Avg Training Acc: 82.750000
Val_: Total Validation Loss: 2.063346, Acc: 19.7625
Test_: Total Testing Acc: 19.0375
Time taken = 118.8459 s

Train: Epoch:45, Batch:19, Loss:1.459424, Acc:81.2500
Train: Epoch:45, Batch:39, Loss:1.450175, Acc:81.2500
Train: Epoch:45, Batch:59, Loss:1.445661, Acc:81.2500
Train: Epoch:45, Batch:79, Loss:1.382321, Acc:87.5000
Train: Epoch:45, Batch:99, Loss:1.476001, Acc:81.2500
Train: Epoch:45, Batch:119, Loss:1.389748, Acc:90.6250
Train: Epoch:45, Batch:139, Loss:1.496939, Acc:75.0000
Train_: Avg Training Loss: 1.447447, Avg Training Acc: 83.062500
Val_: Total Validation Loss: 2.078680, Acc: 18.1875
Test_: Total Testing Acc: 18.3000
Time taken = 118.4148 s

Train: Epoch:46, Batch:19, Loss:1.513622, Acc:75.0000
Train: Epoch:46, Batch:39, Loss:1.411008, Acc:87.5000
Train: Epoch:46, Batch:59, Loss:1.373805, Acc:90.6

Train: Epoch:59, Batch:79, Loss:1.348247, Acc:93.7500
Train: Epoch:59, Batch:99, Loss:1.557687, Acc:71.8750
Train: Epoch:59, Batch:119, Loss:1.357407, Acc:93.7500
Train: Epoch:59, Batch:139, Loss:1.502296, Acc:78.1250
Train_: Avg Training Loss: 1.418369, Avg Training Acc: 85.625000
Val_: Total Validation Loss: 2.056662, Acc: 20.5250
Test_: Total Testing Acc: 20.6750
Time taken = 107.5864 s

Train: Epoch:60, Batch:19, Loss:1.440127, Acc:84.3750
Train: Epoch:60, Batch:39, Loss:1.400825, Acc:87.5000
Train: Epoch:60, Batch:59, Loss:1.391581, Acc:87.5000
Train: Epoch:60, Batch:79, Loss:1.350775, Acc:93.7500
Train: Epoch:60, Batch:99, Loss:1.491004, Acc:78.1250
Train: Epoch:60, Batch:119, Loss:1.336337, Acc:93.7500
Train: Epoch:60, Batch:139, Loss:1.478992, Acc:78.1250
Train_: Avg Training Loss: 1.414793, Avg Training Acc: 86.145833
Val_: Total Validation Loss: 2.052774, Acc: 20.8625
Test_: Total Testing Acc: 21.4625
Time taken = 105.4130 s

Train: Epoch:61, Batch:19, Loss:1.461575, Acc:81.2

Train: Epoch:74, Batch:39, Loss:1.398319, Acc:87.5000
Train: Epoch:74, Batch:59, Loss:1.375390, Acc:90.6250
Train: Epoch:74, Batch:79, Loss:1.310014, Acc:96.8750
Train: Epoch:74, Batch:99, Loss:1.491537, Acc:78.1250
Train: Epoch:74, Batch:119, Loss:1.336311, Acc:93.7500
Train: Epoch:74, Batch:139, Loss:1.489761, Acc:78.1250
Train_: Avg Training Loss: 1.401990, Avg Training Acc: 87.229167
Val_: Total Validation Loss: 2.070880, Acc: 19.2000
Test_: Total Testing Acc: 19.9250
Time taken = 113.9509 s

Train: Epoch:75, Batch:19, Loss:1.429232, Acc:84.3750
Train: Epoch:75, Batch:39, Loss:1.415088, Acc:87.5000
Train: Epoch:75, Batch:59, Loss:1.434923, Acc:84.3750
Train: Epoch:75, Batch:79, Loss:1.303682, Acc:96.8750
Train: Epoch:75, Batch:99, Loss:1.469593, Acc:81.2500
Train: Epoch:75, Batch:119, Loss:1.342477, Acc:93.7500
Train: Epoch:75, Batch:139, Loss:1.497595, Acc:78.1250
Train_: Avg Training Loss: 1.401554, Avg Training Acc: 87.208333
Val_: Total Validation Loss: 2.051777, Acc: 21.2000
T

Test_: Total Testing Acc: 19.8250
Time taken = 128.4285 s

Train: Epoch:89, Batch:19, Loss:1.428014, Acc:84.3750
Train: Epoch:89, Batch:39, Loss:1.431858, Acc:84.3750
Train: Epoch:89, Batch:59, Loss:1.370470, Acc:90.6250
Train: Epoch:89, Batch:79, Loss:1.274874, Acc:100.0000
Train: Epoch:89, Batch:99, Loss:1.457422, Acc:81.2500
Train: Epoch:89, Batch:119, Loss:1.350985, Acc:93.7500
Train: Epoch:89, Batch:139, Loss:1.459003, Acc:81.2500
Train_: Avg Training Loss: 1.384280, Avg Training Acc: 89.104167
Val_: Total Validation Loss: 2.070289, Acc: 19.2750
Test_: Total Testing Acc: 20.0375
Time taken = 110.8636 s

Train: Epoch:90, Batch:19, Loss:1.401578, Acc:87.5000
Train: Epoch:90, Batch:39, Loss:1.447676, Acc:84.3750
Train: Epoch:90, Batch:59, Loss:1.430818, Acc:84.3750
Train: Epoch:90, Batch:79, Loss:1.275807, Acc:100.0000
Train: Epoch:90, Batch:99, Loss:1.469907, Acc:78.1250
Train: Epoch:90, Batch:119, Loss:1.418828, Acc:84.3750
Train: Epoch:90, Batch:139, Loss:1.460797, Acc:81.2500
Tra

#### Test on Center Single, 2x2 Grid, Left-Right, Out-In

In [ ]:
testset = dataset(args.path, "test", args.img_size, transform=transforms.Compose([ToTensor()]))

In [ ]:
center_single_fnames = [x for x in testset.file_names if '/center_single/' in x]
distribute_four_fnames = [x for x in testset.file_names if '/distribute_four/' in x]
in_distribute_four_out_center_single_fnames = \
    [x for x in testset.file_names if '/in_distribute_four_out_center_single/' in x]
left_center_single_right_center_single_fnames = \
    [x for x in testset.file_names if '/left_center_single_right_center_single/' in x]

In [ ]:
print(len(center_single_fnames), \
len(distribute_four_fnames), \
len(in_distribute_four_out_center_single_fnames), \
len(left_center_single_right_center_single_fnames))

2000 2000 2000 2000


In [ ]:
cs_set = dataset(args.path, "test", args.img_size, transform=transforms.Compose([ToTensor()]))
cs_set.file_names = center_single_fnames
cs_testloader = DataLoader(cs_set, batch_size=args.batch_size, shuffle=False, num_workers=80)

df_set = dataset(args.path, "test", args.img_size, transform=transforms.Compose([ToTensor()]))
df_set.file_names = distribute_four_fnames
df_testloader = DataLoader(df_set, batch_size=args.batch_size, shuffle=False, num_workers=80)

idfo_set = dataset(args.path, "test", args.img_size, transform=transforms.Compose([ToTensor()]))
idfo_set.file_names = in_distribute_four_out_center_single_fnames
idfo_testloader = DataLoader(idfo_set, batch_size=args.batch_size, shuffle=False, num_workers=80)

lcsr_set = dataset(args.path, "test", args.img_size, transform=transforms.Compose([ToTensor()]))
lcsr_set.file_names = left_center_single_right_center_single_fnames
lcsr_testloader = DataLoader(lcsr_set, batch_size=args.batch_size, shuffle=False, num_workers=80)

In [ ]:
def test_sep(epoch, tl, save_file):
    model.eval()
    accuracy = 0

    acc_all = 0.0
    counter = 0
    for batch_idx, (image, target, meta_target, meta_structure, embedding, indicator) in enumerate(tl):
        counter += 1
        if args.cuda:
            image = image.cuda()
            target = target.cuda()
            meta_target = meta_target.cuda()
            meta_structure = meta_structure.cuda()
            embedding = embedding.cuda()
            indicator = indicator.cuda()
        acc = model.test_(image, target, meta_target, meta_structure, embedding, indicator)
        # print('Test: Epoch:{}, Batch:{}, Acc:{:.4f}.'.format(epoch, batch_idx, acc))  
        acc_all += acc
    if counter > 0:
        save_str = "Test_: Total Testing Acc: {:.4f}".format(acc_all / float(counter))
        print(save_str)
        with open(save_file, 'a') as f:
            f.write(save_str + "\n")
    return acc_all/float(counter)

In [ ]:
SAVE_FILE_SEP = SAVE_FILE + "_sep"

test_acc_cs = test_sep(epoch, cs_testloader, SAVE_FILE_SEP)
test_acc_df = test_sep(epoch, df_testloader, SAVE_FILE_SEP)
test_acc_idfo = test_sep(epoch, idfo_testloader, SAVE_FILE_SEP)
test_acc_lcsr = test_sep(epoch, lcsr_testloader, SAVE_FILE_SEP)

Test_: Total Testing Acc: 40.0298
Test_: Total Testing Acc: 31.9444
Test_: Total Testing Acc: 46.1806
Test_: Total Testing Acc: 47.5694
